# Nordrhein-Westfalen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Nordrhein-Westfalen is `DEA`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
import patoolib
from glob import glob
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import StringIO

from camelsp import Bundesland


/home/alexander/anaconda3/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/alexander/anaconda3/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('NRW').input_path
BASE

'/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'

In [3]:
# remove everything from BASE folder except Q&W.rar to ensure that data is extracted freshly and is up to date
for f in glob(f"{BASE}/*[!*.rar]"):
    os.remove(f)

# extract rar archive, make sure that unrar is installed
patoolib.extract_archive(f"{BASE}/Q&W.rar", outdir=BASE)

patool: Extracting /home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Q&W.rar ...
patool: running /usr/bin/unrar x -- /home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Q&W.rar
patool:     with cwd='/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'
patool: ... /home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Q&W.rar extracted to `/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'.


'/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen'

In [41]:
# the file Datenanfrage_CAMELS_2581119000100_20221107_140905.csv exists twice, remove (2)
for f in glob(f"{BASE}/Datenanfrage_CAMELS_2581119000100_20221107_140905*"):
    print(f)

os.remove(f"{BASE}/Datenanfrage_CAMELS_2581119000100_20221107_140905 (2).csv")

/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Datenanfrage_CAMELS_2581119000100_20221107_140905 (2).csv
/home/alexander/Github/camels/camelsp/input_data/NRW_Nordrhein-Westfalen/Datenanfrage_CAMELS_2581119000100_20221107_140905.csv


In [104]:
with Bundesland('NRW') as bl:
    metadata = pd.read_excel(os.path.join(bl.input_path, 'Stammdaten_CAMELS.xlsx'))

metadata

,NAME,ORT,NULLPUNKT,STATIONIER,GEBFLAECHE,UTMZONE,KOORDX,KOORDY,KOMMENTAR,GEBIETSKEN,Gewässerkennzahl,Gewässer
0,Ahmsen,4639000000100,64.285,27.158,593.00,32U,479549.677600,5.771202e+06,Grundmessstelle des Landes (GL),4639.0,46,Werre
1,Albersloh,3259000000100,48.678,27.470,321.58,32U,412463.350631,5.748891e+06,NaN,3259.0,32,Werse
2,Altena,2766930000100,154.225,29.700,1190.00,32U,407683.711900,5.682847e+06,Talsperrenbeeinflussung ab 1968 (Biggetalsperre),276693.0,2766,Lenne
3,Altenbeken 2,2781610000200,215.958,11.990,20.50,32U,494359.426900,5.734473e+06,Grundmeßstelle des Landes (GL),278161.0,27816,Beke
4,Altenburg 1,2823900000200,82.651,62.440,958.76,32U,315309.586100,5.641695e+06,Grundmessstelle,28239.0,282,Rur
...,...,...,...,...,...,...,...,...,...,...,...,...
214,Westtuennen,2786700000100,57.572,3.970,414.90,32U,421634.000000,5.724518e+06,Grundmessstelle des Landes (GL),NaN,2786,Ahse
215,Wetter_Wengern_1,2769169000100,84.805,0.480,17.58,32U,384656.787100,5.695710e+06,Grundmeßstelle des Landes (GL),2769169.0,276916,Elbsche
216,Wettringen B70,9286291000300,41.504,6.330,175.07,32U,385480.262390,5.786270e+06,NaN,928629.0,92862,Steinfurter Aa
217,Wt-Kluserbrücke,2736510000100,142.226,49.240,337.82,32U,371494.000000,5.679856e+06,Durch mehrere Talsperren beeinflusst. Seit 01....,273651.0,2736,Wupper


In [59]:
metadata[metadata['ORT'] == 2581119000100]

,NAME,ORT,NULLPUNKT,STATIONIER,GEBFLAECHE,UTMZONE,KOORDX,KOORDY,KOMMENTAR,GEBIETSKEN,Gewässerkennzahl,Gewässer
50,Feudingen,2581119000100,388.738,222.32,25.4,32U,452579.3848,5.643232e+06,NaN,2581119.0,258,Lahn


In [18]:
for map in bl.nuts_mapping:
    if map['provider_id'] == '2581119000100':
        print(map)

{'nuts_id': 'DEA10390', 'provider_id': '2581119000100', 'path': './DEA/DEA10390/DEA10390_data.csv'}
{'nuts_id': 'DEA10470', 'provider_id': '2581119000100', 'path': './DEA/DEA10470/DEA10470_data.csv'}
{'nuts_id': 'DEA10670', 'provider_id': '2581119000100', 'path': './DEA/DEA10670/DEA10670_data.csv'}


In [19]:
found = 0
nope = 0

for map in bl.nuts_mapping:
    try: 
        bl.get_data(map['nuts_id'])
        found += 1
    except:
        nope += 1

print(f"found: {found}, nope: {nope}")

found: 219, nope: 218


### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [97]:
# the id column will be ORT
id_column = 'ORT'

## file extract and parse

Here, we need to process the filename as the `'Ort'` is contained in the filename. Looks like the metadata header is **always** to line 32, indicating a finished header by `YTYP;`. Verify this.

In [21]:
for fname in glob(os.path.join(BASE, 'Datenanfrage_CAMELS_*')):
    df = pd.read_csv(fname, encoding='latin1', sep=';', usecols=[0,1], nrows=32, header=None)
    if df.iloc[31, 0] != 'YTYP':
        print(fname)
        

That's will make our lifes way easier. Now go for all:

In [22]:
# get all file names
filelist = glob(os.path.join(BASE, 'Datenanfrage_CAMELS_*'))

# container for meta-header and dataframes
meta = []
data = []

# go for each file
for fname in tqdm(filelist):
    # open
    with open(fname, 'rb') as f:
        txt = f.read().decode('latin1')
    
    # split header
    header = txt.splitlines()[:32]
    
    # build the meta by hand
    tups = [l.split(';') for l in header[:-1]]
    meta_dict = {t[0]: t[1] for t in tups}

    # check the parameter
    if meta_dict['Parameter'] == 'Wasserstand':
        variable = 'w'
    elif meta_dict['Parameter'] == 'Abfluss':
        variable = 'q'
    else:
        raise RuntimeError(f"Unknown Parameter: {meta_dict['Parameter']}")

    meta.append(meta_dict)
    
    # now get the body
    body = txt.splitlines()[32:]

    # now this stupid check
    second_header = [i for i, l in enumerate(body) if l.startswith('Station')]
    if len(second_header) > 0:
        # THERE IS A SECOND HEADER IN THE FILE !!!! come on!
        body = body[:second_header[0]]
    
    # write to buffer
    buffer = StringIO('\n'.join(body))
    buffer.seek(0)
    
    # read from memory
    df_data = pd.read_csv(buffer, sep=';', usecols=[0,1], skiprows=32, decimal=',', header=None, na_values='LUECKE', parse_dates=[0])
    
    df_data.columns = ['date', variable]
    df_data['flag'] = np.NaN
    
    # append
    data.append(df_data)
    
    
print(f"Parsed {len(meta)} metadata headers and {len(data)} data files")

100%|██████████| 437/437 [4:07:27<00:00, 33.97s/it]      

Parsed 437 metadata headers and 437 data files


That was really stupid. Ok. Check the metadata from the data files:

In [98]:
# create dataframe from meta, drop empty columns
extra = pd.DataFrame(meta).drop(['Gewässer', 'GUELTBIS', 'GUELTVON', 'KOMMENTAR', 'PARMERKMAL', 'XEINHEIT'], axis=1)
extra

,Station,Stationsnummer,Unterbezeichnung,Einzugsgebiet,Pegelnullpunkt,Parameter,Einheit,Aussage,Lebenslauf,Zeitangabe,...,MESSGENAU,NWGRENZE,PUBLIZIERT,QUELLE,REIHENART,VERSION,X,XDISTANZ,XFAKTOR,Y
0,Kreuztal,2721459000100,,"63,40 km²","273,894 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2721459000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,429430,T,1,5645719
1,Liesborn,2784650000100,,"66,30 km²","73,405 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('2784650000100.wk4',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,449139,T,1,5729507
2,Veert,2850000000200,,"0,00 km²","22,376 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('2850000000200.wk2',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,313366,T,1,5710953
3,Raumland,4281490000100,,"84,70 km²","400,254 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('4281490000100.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,456965,T,1,5653501
4,Hörstel,3448390000200,,"88,66 km²","40,018 mDHHN92 (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('3448390000200.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,403784,T,1,5797597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Schleiden,2822870000100,,"159,41 km²","365,278 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2822870000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,320561,T,1,5600047
433,Langenfeld,2721421000100,,"2,89 km²","0,000 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721421000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,437295,T,1,5651320
434,Vossheide,4621900000100,,"82,20 km²","104,552 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('4621900000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,497230,T,1,5762842
435,Niedermehnen,4761270000100,,"62,22 km²","40,110 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('4761270000100.wk4',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,462184,T,1,5806907


Metadata `extra` has different entries for Q and W at the same station. We have to merge these two metadata entries, so that we have one metadata entry for each `Stationsnummer`.

In [99]:
#extra[extra['Stationsnummer']=='2721459000100']

print(f"Two entries for ID '2721459000100':\n{extra[extra['Stationsnummer'] == '2721459000100']}")

# all IDs have two entries -> Q and W
print(f"\nNumber of duplicated IDs: {extra['Stationsnummer'].duplicated().sum()}")

Two entries for ID '2721459000100':
      Station Stationsnummer Unterbezeichnung Einzugsgebiet  \
0    Kreuztal  2721459000100                      63,40 km²   
379  Kreuztal  2721459000100                      63,40 km²   

              Pegelnullpunkt    Parameter Einheit     Aussage  \
0    273,894 mNHN (aktuell)      Abfluss    m³/s  Mittelwert   
379  273,894 mNHN (aktuell)  Wasserstand      cm  Mittelwert   

                         Lebenslauf  \
0     MITTEL('2721459000100.qk0',d)   
379   MITTEL('2721459000100.wk3',d)   

                                           Zeitangabe  ... MESSGENAU NWGRENZE  \
0    Linke Seite des Zeitintervalls mit Intervallwert  ...    0.0000   0.0000   
379  Linke Seite des Zeitintervalls mit Intervallwert  ...    0.0000   0.0000   

    PUBLIZIERT QUELLE REIHENART VERSION       X XDISTANZ XFAKTOR        Y  
0         True      P         Z       0  429430        T       1  5645719  
379       True      P         Z       0  429430        T       1

In [100]:
# get column names where Q and W data differ (at least once)
all_ids = extra['Stationsnummer'].unique()

all_different_cols = []

for id in all_ids:
    dat = extra[extra['Stationsnummer'] == id]
    if len(dat) < 2:
        continue
    different_columns = dat.iloc[0] != dat.iloc[1]
    different_colnames = different_columns[different_columns].index.tolist()
    
    for colname in different_colnames:
        if colname in all_different_cols:
            continue
        else:
            all_different_cols.append(colname)

all_different_cols

['Parameter',
 'Einheit',
 'Lebenslauf',
 'FTOLERANZ',
 'Pegelnullpunkt',
 'QUELLE',
 'PUBLIZIERT',
 'Unterbezeichnung',
 'HOEHE']

merge q and w metadata by adding Q_ and W_ suffixes.

In [101]:
# the following columns have different values in Q and W metadata at least once and therfor get a prefix
all_different_cols = ['Parameter', 'Einheit', 'Lebenslauf', 'FTOLERANZ', 'Pegelnullpunkt', 'QUELLE', 'PUBLIZIERT', 'Unterbezeichnung', 'HOEHE']

# get all unique ids
all_ids = extra['Stationsnummer'].unique()

# container
qw_metadata = []

# go for each id
for id in all_ids:
    if id in ['9284590000100', '4761500000100']:
        # for these two stations we only have w data
        w_meta = extra[(extra['Stationsnummer'] == id) & (extra['Parameter'] == 'Wasserstand')].to_dict(orient='records')[0]
        q_meta = {}
    else:
    # get Q metadata as dict
        q_meta = extra[(extra['Stationsnummer'] == id) & (extra['Parameter'] == 'Abfluss')].to_dict(orient='records')[0]
        # get W metadata as dict
        w_meta = extra[(extra['Stationsnummer'] == id) & (extra['Parameter'] == 'Wasserstand')].to_dict(orient='records')[0]

    q_updated_meta = {}
    w_updated_meta = {}

    # add prefix to Q keys
    for key in q_meta.keys():
        if key in all_different_cols:
            q_updated_meta[f"Q_{key}"] = q_meta[key]
        else:
            q_updated_meta[key] = q_meta[key]
    
    # add prefix to W keys
    for key in w_meta.keys():
        if key in all_different_cols:
            w_updated_meta[f"W_{key}"] = w_meta[key]
        else:
            w_updated_meta[key] = w_meta[key]

    # merge Q and W metadata
    qw_metadata.append({**q_updated_meta, **w_updated_meta})

qw_metadata = pd.DataFrame(qw_metadata)
qw_metadata
    

,Station,Stationsnummer,Q_Unterbezeichnung,Einzugsgebiet,Q_Pegelnullpunkt,Q_Parameter,Q_Einheit,Aussage,Q_Lebenslauf,Zeitangabe,...,Y,W_Unterbezeichnung,W_Pegelnullpunkt,W_Parameter,W_Einheit,W_Lebenslauf,W_FTOLERANZ,W_HOEHE,W_PUBLIZIERT,W_QUELLE
0,Kreuztal,2721459000100,,"63,40 km²","273,894 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2721459000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5645719,,"273,894 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2721459000100.wk3',d)",0.0000,27389,True,P
1,Liesborn,2784650000100,,"66,30 km²","73,405 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2784650000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5729507,,"73,405 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2784650000100.wk4',d)",0.0000,7341,True,P
2,Veert,2850000000200,,"0,00 km²","22,376 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2850000000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5710953,,"22,376 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2850000000200.wk2',d)",0.0100,2238,True,P
3,Raumland,4281490000100,,"84,70 km²","400,254 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('4281490000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5653501,,"400,254 mNHN (aktuell)",Wasserstand,cm,"MITTEL('4281490000100.wk3',d)",0.0000,40025,True,P
4,Hörstel,3448390000200,,"88,66 km²","40,024 mDHHN2016 (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('3448390000200.qk1',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5797597,,"40,018 mDHHN92 (aktuell)",Wasserstand,cm,"MITTEL('3448390000200.wk3',d)",0.0000,4002,True,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Essen-Kettwig,2769800000200,,"20,23 km²","42,333 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2769800000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5691810,,"42,333 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2769800000200.wk3',d)",0.0100,4233,True,P
215,Reifferscheid,2822869000100,,"48,95 km²","416,533 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2822869000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5594459,,"416,533 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2822869000100.wk3',d)",0.0000,41653,True,P
216,Legden Bhf,9286410000100,,"41,43 km²","59,257 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286410000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5767205,,"59,257 mNHN (aktuell)",Wasserstand,cm,"MITTEL('9286410000100.wk0',d)",0.0000,5926,True,F
217,Langenfeld,2721421000100,,"2,89 km²","0,000 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721421000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5651320,,"0,000 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2721421000100.wk3',d)",0.0000,0,True,P


Now we have to left-join the data, as each Stationsnummer exists twice. Thus, it is only the combination of Stationsnummer and variable, that makes the data unique

In [105]:
metadata = qw_metadata.join(metadata.set_index(metadata.ORT.astype(str)), on='Stationsnummer', how='left')
metadata

,Station,Stationsnummer,Q_Unterbezeichnung,Einzugsgebiet,Q_Pegelnullpunkt,Q_Parameter,Q_Einheit,Aussage,Q_Lebenslauf,Zeitangabe,...,NULLPUNKT,STATIONIER,GEBFLAECHE,UTMZONE,KOORDX,KOORDY,KOMMENTAR,GEBIETSKEN,Gewässerkennzahl,Gewässer
0,Kreuztal,2721459000100,,"63,40 km²","273,894 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2721459000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,273.894,9.000,63.40,32U,429429.6850,5.645719e+06,Pegel durch Breitenbachtalsperre beeinflusst,2721459.0,27214,Ferndorf
1,Liesborn,2784650000100,,"66,30 km²","73,405 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2784650000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,73.405,30.400,66.30,32U,449139.0000,5.729507e+06,"Verdichtungsmeßstelle des Landes, Bemerkung :P...",278465.0,27846,Liese
2,Veert,2850000000200,,"0,00 km²","22,376 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2850000000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,22.376,12.830,0.00,32U,313366.0000,5.710953e+06,NaN,28541.0,2854,Nierskanal
3,Raumland,4281490000100,,"84,70 km²","400,254 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('4281490000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,400.254,0.180,84.70,32U,456964.7140,5.653501e+06,NaN,428149.0,42814,Odeborn
4,Hörstel,3448390000200,,"88,66 km²","40,024 mDHHN2016 (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('3448390000200.qk1',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,40.024,10.540,88.66,32U,403784.0000,5.797597e+06,NaN,34483.0,3448,Ibbenbürener Aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Essen-Kettwig,2769800000200,,"20,23 km²","42,333 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2769800000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,42.333,0.360,20.23,32U,356021.0000,5.691810e+06,NaN,27698.0,27698,Rinderbach
215,Reifferscheid,2822869000100,,"48,95 km²","416,533 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2822869000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,416.533,3.440,48.95,32U,320854.2646,5.594459e+06,NaN,2822869.0,282286,Reifferscheider Bach
216,Legden Bhf,9286410000100,,"41,43 km²","59,257 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286410000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,59.257,81.730,41.43,32U,368457.5413,5.767205e+06,Beeinflussung durch Wehrsteuerungen und eine g...,NaN,92864,Dinkel
217,Langenfeld,2721421000100,,"2,89 km²","0,000 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721421000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.000,1.040,2.89,32U,437294.5795,5.651320e+06,NaN,2721421.0,272142,Langenfelder Bach


In [109]:
id_column = 'Stationsnummer'

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [121]:
with Bundesland('NRW') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, 'Stationsnummer', overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for m, df in tqdm(zip(meta, data), total=len(meta)):
            # check the meta
            provider_id = str(m['Stationsnummer'])
            bl.save_timeseries(df, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id    provider_id                              path
0  DEA10000  2721459000100  ./DEA/DEA10000/DEA10000_data.csv
1  DEA10010  2784650000100  ./DEA/DEA10010/DEA10010_data.csv
2  DEA10020  2850000000200  ./DEA/DEA10020/DEA10020_data.csv
3  DEA10030  4281490000100  ./DEA/DEA10030/DEA10030_data.csv
4  DEA10040  3448390000200  ./DEA/DEA10040/DEA10040_data.csv


100%|██████████| 437/437 [00:16<00:00, 26.78it/s]
